In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('All libraries imported successfully.')

All libraries imported successfully.


In [2]:
df_l = pd.read_csv('london_result.csv')
df_l.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,SE2,"Bexley, Greenwich",Abbey Wood,51.451242,0.065413
1,W3,"Ealing, Hammersmith and Fulham",Acton,51.491537,-0.214971
2,CR0,Croydon,Addington,51.371305,-0.101957
3,CR0,Croydon,Addiscombe,51.371305,-0.101957
4,DA5,Bexley,Albany Park,51.441679,0.150488
5,IG2,Redbridge,Aldborough Hatch,51.576253,0.045410
6,EC3,City,Aldgate,51.515618,-0.091998
7,WC2,Westminster,Aldwych,51.497321,-0.137149
8,HA0,Brent,Alperton,39.054783,-84.433552
9,SE20,Bromley,Anerley,51.402805,0.014814


In [3]:
from geopy.geocoders import Nominatim
import folium
address = 'London'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
london_map = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(df_l['Latitude'], df_l['Longitude'], df_l['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(london_map)
london_map

In [4]:
LN = df_l
LN = LN.reset_index(drop=True)
LN.head(20)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,SE2,"Bexley, Greenwich",Abbey Wood,51.451242,0.065413
1,W3,"Ealing, Hammersmith and Fulham",Acton,51.491537,-0.214971
2,CR0,Croydon,Addington,51.371305,-0.101957
3,CR0,Croydon,Addiscombe,51.371305,-0.101957
4,DA5,Bexley,Albany Park,51.441679,0.150488
5,IG2,Redbridge,Aldborough Hatch,51.576253,0.045410
6,EC3,City,Aldgate,51.515618,-0.091998
7,WC2,Westminster,Aldwych,51.497321,-0.137149
8,HA0,Brent,Alperton,39.054783,-84.433552
9,SE20,Bromley,Anerley,51.402805,0.014814


In [5]:
CLIENT_ID = 'FQQVZBYKAWIY21EZBUK1VS5YA50X2DTW13LFXZ5EEDGDHLNS'
CLIENT_SECRET = 'LOZ00NH1JID2FA2OJ2OLHSGHCI0HSVZ4RDKORBPPSXLGTQ4Q'
VERSION = '20180605'
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: FQQVZBYKAWIY21EZBUK1VS5YA50X2DTW13LFXZ5EEDGDHLNS
CLIENT_SECRET:LOZ00NH1JID2FA2OJ2OLHSGHCI0HSVZ4RDKORBPPSXLGTQ4Q


In [6]:
address = 'London'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
LN_map = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(LN['Latitude'], LN['Longitude'], LN['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(LN_map)
LN_map

In [7]:
LIMIT=100
radius=500
def getNearbyVenues(names, latitudes, longitudes, radius=500):   
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_ven = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_ven.columns = ['Neighborhood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_ven)

In [8]:
LN_ven = getNearbyVenues(names=LN['Neighborhood'],
                                   latitudes=LN['Latitude'],
                                   longitudes=LN['Longitude']
                                  )

In [9]:
LN_ven.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbey Wood,51.451242,0.065413,Eltham Park South,51.455424,0.066433,Park
1,Abbey Wood,51.451242,0.065413,Star Of India,51.450714,0.064183,Indian Restaurant
2,Abbey Wood,51.451242,0.065413,Prezzo,51.450789,0.058403,Italian Restaurant
3,Abbey Wood,51.451242,0.065413,China Noodle,51.450297,0.058845,Chinese Restaurant
4,Abbey Wood,51.451242,0.065413,Kaspas,51.450626,0.058591,Dessert Shop
5,Acton,51.491537,-0.214971,St Paul's Hotel,51.493639,-0.215494,Hotel
6,Acton,51.491537,-0.214971,The Melody Restaurant,51.493529,-0.215360,Cocktail Bar
7,Acton,51.491537,-0.214971,Latymers,51.493110,-0.216629,Pub
8,Acton,51.491537,-0.214971,L'Elysee,51.494967,-0.211737,Bakery
9,Acton,51.491537,-0.214971,Queen's Club,51.487360,-0.212558,Tennis Stadium


In [11]:
Neighbor_lt = list(LN_ven['Neighborhood'].unique())
print('Number of Neighborhoods inside London: ')
print(len(Neighbor_lt))
print('List of Neighborhoods:')
Neighbor_lt

Number of Neighborhoods inside London: 
509
List of Neighborhoods:


['Abbey Wood',
 'Acton',
 'Addington',
 'Addiscombe',
 'Albany Park',
 'Aldborough Hatch',
 'Aldgate',
 'Aldwych',
 'Alperton',
 'Anerley',
 'Angel',
 'Aperfield',
 'Archway',
 'Ardleigh Green',
 'Arkley',
 'Arnos Grove',
 'Bankside',
 'Barbican',
 'Barking',
 'Barkingside',
 'Barnehurst',
 'Barnes',
 'Barnes Cray',
 'Barnet Gate',
 'Barnet?(also Chipping Barnet, High Barnet)',
 'Barnsbury',
 'Bayswater',
 'Beckenham',
 'Beckton',
 'Becontree',
 'Becontree Heath',
 'Beddington',
 'Bedford Park',
 'Belgravia',
 'Bellingham',
 'Belmont',
 'Belsize Park',
 'Belvedere',
 'Bermondsey',
 'Berrylands',
 'Bethnal Green',
 'Bexley?(also Old Bexley, Bexley Village)',
 'Bexleyheath(also Bexley New Town)',
 'Bickley',
 'Biggin Hill',
 'Blackfen',
 'Blackfriars',
 'Blackheath',
 'Blackheath Royal Standard',
 'Blackwall',
 'Blendon',
 'Bloomsbury',
 'Botany Bay',
 'Bounds Green',
 'Bow',
 'Bowes Park',
 'Brentford',
 'Brent Cross',
 'Brent Park',
 'Brimsdown',
 'Brixton',
 'Brockley',
 'Bromley',
 '

In [12]:
Neighbor_ven_sum = LN_ven.groupby('Neighborhood').count()
Neighbor_ven_sum.head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abbey Wood,5,5,5,5,5,5
Acton,28,28,28,28,28,28
Addington,27,27,27,27,27,27
Addiscombe,27,27,27,27,27,27
Albany Park,9,9,9,9,9,9
Aldborough Hatch,5,5,5,5,5,5
Aldgate,100,100,100,100,100,100
Aldwych,74,74,74,74,74,74
Alperton,1,1,1,1,1,1


In [13]:
print('There are {} uniques categories.'.format(len(LN_ven['Venue Category'].unique())))
print('The list of different categories:')
list(LN_ven['Venue Category'].unique())

There are 232 uniques categories.
The list of different categories:


['Park',
 'Indian Restaurant',
 'Italian Restaurant',
 'Chinese Restaurant',
 'Dessert Shop',
 'Hotel',
 'Cocktail Bar',
 'Pub',
 'Bakery',
 'Tennis Stadium',
 'Japanese Restaurant',
 'Sandwich Place',
 'Indie Theater',
 'Garden',
 'Bar',
 'Café',
 'Gym',
 'Gym / Fitness Center',
 'Coffee Shop',
 'Pizza Place',
 'Thai Restaurant',
 'Grocery Store',
 'Bus Stop',
 'Costume Shop',
 'Gaming Cafe',
 'Bookstore',
 'Portuguese Restaurant',
 'Burger Joint',
 'Spanish Restaurant',
 'Nightclub',
 'Caribbean Restaurant',
 'Malay Restaurant',
 'Mediterranean Restaurant',
 'Clothing Store',
 'Sushi Restaurant',
 'Asian Restaurant',
 'Furniture / Home Store',
 'Insurance Office',
 'Tram Station',
 'Greek Restaurant',
 'Fast Food Restaurant',
 'Train Station',
 'Toy / Game Store',
 'Metro Station',
 'Eastern European Restaurant',
 'Steakhouse',
 'Vietnamese Restaurant',
 'Art Gallery',
 'French Restaurant',
 'Roof Deck',
 'Yoga Studio',
 'Scenic Lookout',
 'Seafood Restaurant',
 'Butcher',
 'Lounge',

In [14]:
LN_cate = pd.get_dummies(data = LN_ven, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
LN_cate.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,ATM,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Bike Rental / Bike Share,Bistro,Boarding House,Bookstore,Border Crossing,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Canal,Candy Store,Caribbean Restaurant,Casino,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Optical Shop,Organic Grocery,Outdoor Supply Store,Outlet Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pharmacy,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toll Plaza,Tour Provider,Toy / Game Store,Trail,Train Station,Tram Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Abbey Wood,51.451242,0.065413,Eltham Park South,51.455424,0.066433,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Abbey Wood,51.451242,0.065413,Star Of India,51.450714,0.

In [17]:
im_fe = [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',

 'Park',
 'Dessert Shop',
 'Hotel',
 'Cocktail Bar',
 'Pub',
 'Bakery',
 'Tennis Stadium',
 'Sandwich Place',
 'Indie Theater',
 'Garden',
 'Bar',
 'Café',
 'Gym',
 'Gym / Fitness Center',
 'Coffee Shop',
 'Grocery Store',
 'Bus Stop',
 'Costume Shop',
 'Gaming Cafe',
 'Bookstore',
 'Burger Joint',
 'Nightclub',
 'Clothing Store',
 'Furniture / Home Store',
 'Insurance Office',
 'Tram Station',
 'Train Station',
 'Toy / Game Store',
 'Metro Station',
 'Art Gallery',
 'Roof Deck',
 'Yoga Studio',
 'Scenic Lookout',
 'Butcher',
 'Lounge',
 'History Museum',
 'BBQ Joint',
 'Plaza',
 'Performing Arts Venue',
 'Event Space',
 'Historic Site',
 'Concert Hall',
 'Theater',
 'Wine Bar',
 'Botanical Garden',
 'Boxing Gym',
 'Indie Movie Theater',
 'Burrito Place',
 'Tea Room',
 'Salad Place',
 'Museum',
 'Shopping Mall',
 'Cycle Studio',
 'Breakfast Spot',
 'Juice Bar',
 'Movie Theater',
 'Sporting Goods Shop',
 'Hotel Bar',
 'Chocolate Shop',
 'Supermarket',
 'Pedestrian Plaza',
 'Poke Place',
 'Gift Shop',
 'Playground',
 'Donut Shop',
 'Border Crossing',
 'Ice Cream Shop',
 'Electronics Store',
 'Department Store',
 'Cosmetics Shop',
 'Irish Pub',
 'Stationery Store',
 "Women's Store",
 'Noodle House',
 'Fish Market',
 'Hobby Shop',
 'Organic Grocery',
 'Boutique',
 'Arts & Crafts Store',
 'Beer Bar',
 'Record Shop',
 'Gastropub',
 'Tiki Bar',
 'Bistro',
 'Dive Bar',
 'Antique Shop',
 'Music Venue',
 'Cheese Shop',
 'Souvlaki Shop',
 'Fried Chicken Joint',
 'Comedy Club',
 'Spa',
 'Road',
 'Thrift / Vintage Store',
 'Dance Studio',
 'Bike Rental / Bike Share',
 'Bed & Breakfast',
 'Optical Shop',
 'Pharmacy',
 'Video Game Store',
 'Mobile Phone Shop',
 'Shoe Store',
 'Outdoor Supply Store',
 'River',
 'Gay Bar',
 'Liquor Store',
 'Bus Station',
 'Garden Center',
 'Trail',
 'Bagel Shop',
 'Warehouse Store',
 'Discount Store',
 'Hookah Bar',
 'Massage Studio',
 'Platform',
 'Food Truck',
 'Market',
 'Light Rail Station',
 'Baby Store',
 'Bank',
 'Convenience Store',
 'Toll Plaza',
 'Public Art',
 'Kitchen Supply Store',
 'Smoothie Shop',
 'Hardware Store',
 'Candy Store',
 'Lingerie Store',
 'Outlet Store',
 'Street Art',
 'Skate Park',
 'General Entertainment',
 'Food & Drink Shop',
 'Aquarium',
 'Arcade',
 'Farmers Market',
 'Champagne Bar',
 'Frozen Yogurt Shop',
 'Residential Building (Apartment / Condo)',
 'Science Museum',
 'Exhibit',
 'Casino',
 'Planetarium',
 'Tour Provider',
 'Shopping Plaza',
 'Farm',
 'Flea Market',
 'Cricket Ground',
 'Snack Place',
 'Cupcake Shop',
 'Indoor Play Area',
 'Hockey Field',
 "Men's Store",
 'Brewery',
 'Lake',
 'Creperie',
 'Chaat Place',
 'Multiplex',
 'Bubble Tea Shop',
 'Wings Joint',
 'Boarding House',
 'Hostel',
 'Rental Car Location',
 'Jazz Club',
 'Sports Bar',
 'Wine Shop',
 'Building',
 'Pie Shop',
 'Field',
 'Canal',
 'Athletics & Sports',
 'Shipping Store',
 'ATM',
 'Laser Tag']

In [18]:
LN_hs = LN_cate[im_fe].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()
LN_hs.head(10)

,Park,Dessert Shop,Hotel,Cocktail Bar,Pub,Bakery,Tennis Stadium,Sandwich Place,Indie Theater,Garden,Bar,Café,Gym,Gym / Fitness Center,Coffee Shop,Grocery Store,Bus Stop,Costume Shop,Gaming Cafe,Bookstore,Burger Joint,Nightclub,Clothing Store,Furniture / Home Store,Insurance Office,Tram Station,Train Station,Toy / Game Store,Metro Station,Art Gallery,Roof Deck,Yoga Studio,Scenic Lookout,Butcher,Lounge,History Museum,BBQ Joint,Plaza,Performing Arts Venue,Event Space,Historic Site,Concert Hall,Theater,Wine Bar,Botanical Garden,Boxing Gym,Indie Movie Theater,Burrito Place,Tea Room,Salad Place,Museum,Shopping Mall,Cycle Studio,Breakfast Spot,Juice Bar,Movie Theater,Sporting Goods Shop,Hotel Bar,Chocolate Shop,Supermarket,Pedestrian Plaza,Poke Place,Gift Shop,Playground,Donut Shop,Border Crossing,Ice Cream Shop,Electronics Store,Department Store,Cosmetics Shop,Irish Pub,Stationery Store,Women's Store,Noodle House,Fish Market,Hobby Shop,Organic Grocery,Boutique,Arts & Crafts Store,Beer Bar,Record Shop,Gastropub,Tiki Bar,Bistro,Dive Bar,Antique Shop,Music Venue,Cheese Shop,Souvlaki Shop,Fried Chicken Joint,Comedy Club,Spa,Road,Thrift / Vintage Store,Dance Studio,Bike Rental / Bike Share,Bed & Breakfast,Optical Shop,Pharmacy,Video Game Store,Mobile Phone Shop,Shoe Store,Outdoor Supply Store,River,Gay Bar,Liquor Store,Bus Station,Garden Center,Trail,Bagel Shop,Warehouse Store,Discount Store,Hookah Bar,Massage Studio,Platform,Food Truck,Market,Light Rail Station,Baby Store,Bank,Convenience Store,Toll Plaza,Public Art,Kitchen Supply Store,Smoothie Shop,Hardware Store,Candy Store,Lingerie Store,Outlet Store,Street Art,Skate Park,General Entertainment,Food & Drink Shop,Aquarium,Arcade,Farmers Market,Champagne Bar,Frozen Yogurt Shop,Residential Building (Apartment / Condo),Science Museum,Exhibit,Casino,Planetarium,Tour Provider,Shopping Plaza,Farm,Flea Market,Cricket Ground,Snack Place,Cupcake Shop,Indoor Play Area,Hockey Field,Men's Store,Brewery,Lake,Creperie,Chaat Place,Multiplex,Bubble Tea Shop,Wings Joint,Boarding House,Hostel,Rental Car Location,Jazz Club,Sports Bar,Wine Shop,Building,Pie Shop,Field,Canal,Athletics & Sports,Shipping Store,ATM,Laser Tag
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abbey Wood,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Acton,0,0,3,2,3,1,1,1,1,1,1,1,2,1,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Addington,0,0,0,0,3,0,0,0,0,0,0,1,0,1,4,0,0,0,1,1,2,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Addiscombe,0,0,0,0,3,0,0,0,0,0,0,1,0,1,4,0,0,0,1,1,2,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Albany Park,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [19]:
feat_ne_lt = list(LN_hs.columns)
store_lt = []

for counter, value in enumerate(feat_ne_lt):
    if value.find('Store') != (-1):
        store_lt.append(value)
        
LN_hs['Total Stores'] = LN_hs[store_lt].sum(axis = 1)
LN_hs = LN_hs.drop(columns = store_lt)

feat_ne_lt = list(LN_hs.columns)
shop_lt = []

for counter, value in enumerate(feat_ne_lt):
    if value.find('Shop') != (-1):
        shop_lt.append(value)
        
LN_hs['Total Shops'] = LN_hs[shop_lt].sum(axis = 1)
LN_hs = LN_hs.drop(columns = shop_lt)
LN_hs.head(20)

,Park,Hotel,Cocktail Bar,Pub,Bakery,Tennis Stadium,Sandwich Place,Indie Theater,Garden,Bar,Café,Gym,Gym / Fitness Center,Bus Stop,Gaming Cafe,Bookstore,Burger Joint,Nightclub,Insurance Office,Tram Station,Train Station,Metro Station,Art Gallery,Roof Deck,Yoga Studio,Scenic Lookout,Butcher,Lounge,History Museum,BBQ Joint,Plaza,Performing Arts Venue,Event Space,Historic Site,Concert Hall,Theater,Wine Bar,Botanical Garden,Boxing Gym,Indie Movie Theater,Burrito Place,Tea Room,Salad Place,Museum,Cycle Studio,Breakfast Spot,Juice Bar,Movie Theater,Hotel Bar,Supermarket,Pedestrian Plaza,Poke Place,Playground,Border Crossing,Irish Pub,Noodle House,Fish Market,Organic Grocery,Boutique,Beer Bar,Gastropub,Tiki Bar,Bistro,Dive Bar,Music Venue,Fried Chicken Joint,Comedy Club,Spa,Road,Dance Studio,Bike Rental / Bike Share,Bed & Breakfast,Pharmacy,River,Gay Bar,Bus Station,Garden Center,Trail,Hookah Bar,Massage Studio,Platform,Food Truck,Market,Light Rail Station,Bank,Toll Plaza,Public Art,Street Art,Skate Park,General Entertainment,Aquarium,Arcade,Farmers Market,Champagne Bar,Residential Building (Apartment / Condo),Science Museum,Exhibit,Casino,Planetarium,Tour Provider,Farm,Flea Market,Cricket Ground,Snack Place,Indoor Play Area,Hockey Field,Brewery,Lake,Creperie,Chaat Place,Multiplex,Wings Joint,Boarding House,Hostel,Rental Car Location,Jazz Club,Sports Bar,Building,Field,Canal,Athletics & Sports,ATM,Laser Tag,Total Stores,Total Shops
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abbey Wood,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Acton,0,3,2,3,1,1,1,1,1,1,1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3
Addington,0,0,0,3,0,0,0,0,0,0,1,0,1,0,1,1,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,4
Addiscombe,0,0,0,3,0,0,0,0,0,0,1,0,1,0,1,1,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,4
Albany Park,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
Aldborough Hatch,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Aldgate,2,2,2,2,1,0,1,0,1,1,2,0,2,0,0,1,0,0,0,0,0,0,3,1,2,3,1,1,2,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,13
Aldwych,1,8,0,2,0,0,8,0,0,0,2,1,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,4,1,0,0,0,0,1,0,0,0,1,2,1,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,12
Alperton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 40, random_state = 0).fit(LN_hs)

In [64]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = LN_hs.columns
means_df.index = ['G1','G2','G3','G4','G5','G6','G7','G8','G9','G10','G11','G12','G13','G14','G15','G16','G17','G18','G19','G20','G21','G22','G23','G24','G25','G26','G27','G28','G29','G30','G31','G32','G33','G34','G35','G36','G37','G38','G39','G40']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Park,Hotel,Cocktail Bar,Pub,Bakery,Tennis Stadium,Sandwich Place,Indie Theater,Garden,Bar,Café,Gym,Gym / Fitness Center,Bus Stop,Gaming Cafe,Bookstore,Burger Joint,Nightclub,Insurance Office,Tram Station,Train Station,Metro Station,Art Gallery,Roof Deck,Yoga Studio,Scenic Lookout,Butcher,Lounge,History Museum,BBQ Joint,Plaza,Performing Arts Venue,Event Space,Historic Site,Concert Hall,Theater,Wine Bar,Botanical Garden,Boxing Gym,Indie Movie Theater,Burrito Place,Tea Room,Salad Place,Museum,Cycle Studio,Breakfast Spot,Juice Bar,Movie Theater,Hotel Bar,Supermarket,Pedestrian Plaza,Poke Place,Playground,Border Crossing,Irish Pub,Noodle House,Fish Market,Organic Grocery,Boutique,Beer Bar,Gastropub,Tiki Bar,Bistro,Dive Bar,Music Venue,Fried Chicken Joint,Comedy Club,Spa,Road,Dance Studio,Bike Rental / Bike Share,Bed & Breakfast,Pharmacy,River,Gay Bar,Bus Station,Garden Center,Trail,Hookah Bar,Massage Studio,Platform,Food Truck,Market,Light Rail Station,Bank,Toll Plaza,Public Art,Street Art,Skate Park,General Entertainment,Aquarium,Arcade,Farmers Market,Champagne Bar,Residential Building (Apartment / Condo),Science Museum,Exhibit,Casino,Planetarium,Tour Provider,Farm,Flea Market,Cricket Ground,Snack Place,Indoor Play Area,Hockey Field,Brewery,Lake,Creperie,Chaat Place,Multiplex,Wings Joint,Boarding House,Hostel,Rental Car Location,Jazz Club,Sports Bar,Building,Field,Canal,Athletics & Sports,ATM,Laser Tag,Total Stores,Total Shops,Total Sum
G36,0.000000e+00,3.000000e+00,0.000000e+00,9.000000e+00,2.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,8.000000e+00,0.0,3.000000e+00,0.000000e+00,1.000000e+00,3.000000e+00,4.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1.0,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.700000e+01,2.400000e+01,89.000000
G9,1.000000e+00,4.000000e+00,2.000000e+00,1.000000e+01,2.000000e+00,0.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,1.0,4.000000e+00,5.551115e-17,0.000000e+00,5.551115e-17,1.000000e+00,-1.387779e-17,0.000000e+00,1.387779e-17,0.000000e+00,1.000000e+00,0.000000e+00,8.673617e-19,1.387779e-17,6.938894e-18,0.000000e+00,0.000000e+00,1.734723e-18,0.000000e+00,1.000000e+00,-6.938894e-18,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.387779e-17,8.673617e-19,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,8.673617e-19,2.000000e+00,2.775558e-17,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,-6.938894e-18,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.775558e-17,3.469447e-18,0.000000e+00,3.469447e-18,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.469447e-18,-6.938894e-18,0.000000e+00,0.000000e+00,2.000000e+00,

In [65]:
Neighborhood = pd.DataFrame([LN_hs.index, 1 + kmeans.labels_]).T
Neighborhood.columns = ['Neighborhood', 'Group']
Neighborhood.head()

,Neighborhood,Group
0,Abbey Wood,31
1,Acton,39
2,Addington,2
3,Addiscombe,2
4,Albany Park,17


In [66]:
Neighborhood[Neighborhood['Group'] == 36] #Group 36 is the best group

,Neighborhood,Group
103,Coombe,36


In [67]:
print(LN.loc[LN['Neighborhood'].str.contains('Coombe')])

    Postcode               Borough Neighborhood   Latitude  Longitude
104      CR0               Croydon       Coombe  51.371305  -0.101957
105      KT2  Kingston upon Thames       Coombe  51.409627  -0.306262


The best location is Coombe. Postcode is CR0 (51.371305,-0.101957) or KT2 (51.409627,-0.306262)